This file serves as a class to form the serial connection to an Alicat device.

In [ ]:
gases = ['Air', 'Ar', 'CH4', 'CO', 'CO2', 'C2H6', 'H2', 'He',
                      'N2', 'N2O', 'Ne', 'O2', 'C3H8', 'n-C4H10', 'C2H2',
                      'C2H4', 'i-C2H10', 'Kr', 'Xe', 'SF6', 'C-25', 'C-10',
                      'C-8', 'C-2', 'C-75', 'A-75', 'A-25', 'A1025', 'Star29',
                      'P-5']

In [ ]:
gas = 'Air'
command = '{addr}$${gas}\r'.format(addr='A', gas=gases.index(gas))

In [ ]:
ser.write(command.encode('ascii'))

In [ ]:
ser.write('AR24\r'.encode('ascii'))
read_output(ser,'A','out')


In [ ]:
ser.readline()

In [ ]:
def read_output(ser,ID,output):
    
    if output != 'out':
        command = f'{ID}\r'.encode('ascii')
        ser.write(command)
    
    line = bytearray()
    while True:
        c = ser.read(1)
        if c:
            line += c
            if line[-1] == ord('\r'):
                break
        else:
            break
            
    ser.flush()
    ser.flushInput()
    ser.flushOutput()
    
    return line.decode('ascii').strip()

In [ ]:
def _readline(ser):
        """Read a line using a custom newline character (CR in this case).
        Function from http://stackoverflow.com/questions/16470903/
        pyserial-2-6-specify-end-of-line-in-readline
        """
        

        line = bytearray()
        while True:
            c = ser.read(1)
            if c:
                line += c
                if line[-1] == ord('\r'):
                    break
            else:
                break
        return line.decode('ascii').strip()

In [ ]:
ser.flush()
ser.flushInput()
ser.flushOutput()

In [ ]:
line = bytearray()
while True:
    c = ser.read(1)
    if c:
        line += c
        if line[-1] == ord('\r'):
            break
    else:
        break
    

In [1]:
try:
    import serial
    from dataclasses import dataclass
except ImportError:
    print("An error occurred while attempting to import the pyserial backend. Please check your installation and try again.")


In [2]:
class Serial_Connection(object):
    
    open_ports = {}
    
    def __init__(self, port='/dev/ttyUSB0', baud=19200):
        
        self.port, self.baud = port, baud
        self.devices = {}
        
        if port in Serial_Connection.open_ports:
            self.connection = Serial_Connection.open_ports[port]
        else:
            self.connection = serial.Serial(port, baud, timeout=2.0)
            Serial_Connection.open_ports[port] = self.connection
        
        self.open = True
        
    def _test_open(self):
        
        if not self.open:
            raise IOError(f"The connection to Alicats on port {self.port} not open")
    
    
    def flush(self):
        
        self._test_open()
        
        self.connection.flush()
        self.connection.flushInput()
        self.connection.flushOutput()

    def close(self):
        
        if not self.open:
            return
        
        self.flush()
        
        self.connection.close()
        Serial_Connection.open_ports.pop(self.port, None)
        
        self.open = False
        
    def read(self):
        
        self._test_open()
        
        line = bytearray()
        while True:
            c = self.connection.read(1)
            if c:
                line += c
                if line[-1] == ord('\r'):
                    break
            else:
                break
        
        return line.decode('ascii').strip()
    
    def write(self, ID, command):
        
        command = str(ID) + str(command) + '\r'
        command = command.encode('ascii')
        self.connection.write(command)
        response = []
        response.append(self.read())
        while True:
            if response[-1] != '':
                response.append(self.read())
            else:
                return response[:-1]
            
    def remove_characters(self, array, *kwargs):
        
        if isinstance(array[0], list):
            
            for i in kwargs:
                for j in range(len(array)):
                    array[j] = list(filter((kwargs[i]).__ne__, array[j]))
                    
        else:
            
            for i in kwargs:
                array = list(filter((kwargs[i]).__ne__, array))
        
        return array
    
    
    
    

In [12]:
class MassFlowMeter(Serial_Connection):
    
    def __init__(self, ID :str ='A', port : str ='/dev/ttyUSB0', baud : int =19200):
        super().__init__(port, baud)
        self.ID, self.port, self.baud = ID, port, baud
        self.fetch_gas_list()
        self.fetch_firmware_version()
        
    
    def fetch_gas_list(self):
        self.gas_list = {}
        gases = [i.split() for i in self.write(self.ID, '??G*')]
        gases = self.remove_characters(gases, '\x08')
        for i in range(len(gases)):
            self.gas_list[gases[i][2]] = (gases[i][1])[1:]
        
    
    def fetch_firmware_version(self):
        manufacturer_data = [i.split() for i in super().write(self.ID, '??M*')]
        self.manufacturer_data = self.remove_characters(manufacturer_data,'\x08')
        self.firmware_version = self.manufacturer_data[-1][-1]
        
    def data_format(self):
        pass

In [ ]:
AAA = Serial_Connection('COM5',19200)

In [ ]:
BBB.write('A','??D*')

In [13]:
BBB = MassFlowMeter('A','COM5',19200)

IndexError: list index out of range

In [ ]:
BBB.firmware_version


In [ ]:
BBB.gas_list

In [9]:
BBB.flush()
BBB.close()